In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Train/data/NewFullInsuranceFraudData.csv",low_memory=False)
# df=df.iloc[:,1:]

In [ ]:
# Step 2: Split the DataFrame into two equal parts
mid_index = len(df) // 2
df_part1 = df.iloc[:mid_index].reset_index(drop=True)
df_part2 = df.iloc[mid_index:].reset_index(drop=True)

# Step 3: Save the two parts as separate CSV files
df_part1.to_csv('/content/drive/MyDrive/Train/data//dataset_part1.csv', index=False)
df_part2.to_csv('/content/drive/MyDrive/Train/data//dataset_part2.csv', index=False)

print("Dataset split and saved successfully.")


Dataset split and saved successfully.


In [ ]:
"isFraud" in df.columns.tolist()

False

In [ ]:
print(df.columns.tolist())

['AgeOfVehicle_7 years', 'dataset_name', 'NoOfMonths_PartACov_11', 'NumberOfCars_3 to 4', 'RenalDiseaseIndicator', 'Days:Policy-Accident_1 to 7', 'PolicyType_Sedan - Collision', 'umbrella_limit_0', 'SOCIAL_CLASS_MI', 'MaritalStatus_Single', 'PastNumberOfClaims_more than 4', 'NoOfMonths_PartACov_8', 'AgeOfVehicle_new', 'ChronicCond_Osteoporasis_2', 'witnesses_1', 'DriverRating_1', 'CITY_Nashville', 'number_of_vehicles_involved_1', 'incident_state_SC', 'INCIDENT_CITY_Manchester', 'CUSTOMER_EDUCATION_LEVEL_High School', 'VehiclePrice_20,000 to 29,000', 'incident_city_Arlington', 'STATE_AR', 'ChronicCond_ObstrPulmonary_2', 'insured_hobbies_bungie-jumping', 'AUTHORITY_CONTACTED_Police', 'auto_model_MDX', 'authorities_contacted_Police', 'DayOfWeekClaimed_Monday', 'WeekOfMonth_5', 'INCIDENT_CITY_Glendale', 'insured_education_level_JD', 'insured_relationship_unmarried', 'Police_Report', 'CUSTOMER_EDUCATION_LEVEL_Bachelor', 'MonthClaimed_Mar', 'insured_education_level_Associate', 'IPAnnualReimb

In [ ]:
def display(matching_rows):
    print("counting list of Uniquic all featurs")
    li=[]
    for i in matching_rows.columns:
        #print(str(i),' :',len(matching_rows[i].unique()),matching_rows[i].unique())
        print(str(i), ' :', matching_rows[i].nunique(),',',list(matching_rows[i].unique())[:2])
        li.append(matching_rows[i].nunique())
    return li
        # if(matching_rows[i].nunique()<50):
        #     print(str(i),' :',matching_rows[i].unique())

In [ ]:
display(df)

In [ ]:
df.shape

(159841, 89)

In [ ]:
df['isFraud'].value_counts()/len(df)

,count
isFraud,
0,0.98242
1,0.01758


In [ ]:
a =df.copy()

In [ ]:
df.drop(columns =["TransactionID","dataset_name"],inplace=True)

In [ ]:
# for i in df.columns:
#     print(i)
df_new=df

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

# Custom layer for VAE loss
class VAELossLayer(Layer):
    def __init__(self, **kwargs):
        super(VAELossLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean, z_log_var, z_mean):
        reconstruction_loss = binary_crossentropy(x, x_decoded_mean)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        return K.mean(reconstruction_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_log_var = inputs[2]
        z_mean = inputs[3]

        loss = self.vae_loss(x, x_decoded_mean, z_log_var, z_mean)
        self.add_loss(loss)

        return x

# Data preparation
data_normalized = (df_new - df_new.mean()) / df_new.std()
data_normalized
original_dim = data_normalized.shape[1]

# VAE architecture
input_shape = (original_dim,)
intermediate_dim = 64
latent_dim = 32

# Encoder
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
outputs = VAELossLayer()([inputs, x_decoded_mean, z_log_var, z_mean])
vae = Model(inputs, outputs)

# Compile model
vae.compile(optimizer='rmsprop')

# Train VAE
data_normalized_np = data_normalized.to_numpy()
vae.fit(data_normalized_np, epochs=50, batch_size=32, validation_split=0.1)

# Impute missing values
missing_mask = data_normalized.isnull()
data_filled = data_normalized.fillna(0)
predicted_data = vae.predict(data_filled)
imputed_data = data_normalized.where(~missing_mask, predicted_data)
imputed_data = imputed_data * df_new.std() + df_new.mean()

# Round the imputed values to three decimal places
imputed_data = imputed_data.round(3)

# Display the imputed data
imputed_data

Epoch 1/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 207736864243712.0000 - val_loss: inf
Epoch 2/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 7106179620667392.0000 - val_loss: inf
Epoch 3/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 49063900211573509832059125760.0000 - val_loss: inf
Epoch 4/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 366194064239719267140894720.0000 - val_loss: 8625398839186544830581833728.0000
Epoch 5/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 2262451397211893627277541376.0000 - val_loss: 4578891454459222293479424.0000
Epoch 6/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 503511885378830201304121344.0000 - val_loss: 2996130836975822986805248.0000
Epoch 7/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 3227017920448575241371779072.0000 - val_loss: 2224023196450695615086592.0000
Epoch 8/50
4496/4496 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 392486429928906375573798912.0000 - val_loss: 1888946016687105

,isFraud,transaction_type_EMI Payment,Total ERC20 tnxs,oldbalanceOrg,transaction_type_Deposit,newbalanceOrig,ERC20_most_rec_token_type_Others,merchant_category_Utilities,ERC20 total ether sent,ERC20 most sent token type_Golem,...,ERC20 uniq sent addr.1,ERC20 avg val rec,ERC20 total Ether sent contract,newbalanceDest,oldbalanceDest,ERC20 min val sent,ERC20 most sent token type_Others,ERC20 min val rec,type_CASH_OUT,merchant_category_Entertainment
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159836,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.668,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
159837,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,13.370,0.0,0.0,0.0,0.0,0.0,13.37,0.0,0.0
159838,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,300.694,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
159839,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0


In [ ]:
df.shape

(159841, 87)

In [ ]:
imputed_data.shape

(159841, 87)

In [ ]:
aa=display(df)

counting list of Uniquic all featurs
isFraud  : 2 , [np.int64(0), np.int64(1)]
dataset_name  : 3 , ['Transaction_Fraud', 'Mobile_Transaction_Fraud']
transaction_type_EMI Payment  : 2 , [np.int64(0), np.int64(1)]
 Total ERC20 tnxs  : 300 , [np.int64(0), np.int64(265)]
oldbalanceOrg  : 54092 , [np.float64(0.0), np.float64(19391.0)]
transaction_type_Deposit  : 2 , [np.int64(0), np.int64(1)]
newbalanceOrig  : 42997 , [np.float64(0.0), np.float64(16374.82)]
 ERC20_most_rec_token_type_Others  : 2 , [np.int64(0), np.int64(1)]
merchant_category_Utilities  : 2 , [np.int64(0), np.int64(1)]
 ERC20 total ether sent  : 1415 , [np.float64(0.0), np.float64(35603169.52)]
 ERC20 most sent token type_Golem  : 2 , [np.int64(0), np.int64(1)]
device_used_Web  : 2 , [np.int64(1), np.int64(0)]
TransactionID  : 159841 , ['DigitalTrans_000001', 'DigitalTrans_000002']
transaction_type_Bill Payment  : 2 , [np.int64(1), np.int64(0)]
transaction_type_Loan Disbursement  : 2 , [np.int64(0), np.int64(1)]
merchant_cat

In [ ]:
bb=display(imputed_data)

counting list of Uniquic all featurs
isFraud  : 2 , [np.float64(0.0), np.float64(1.0)]
transaction_type_EMI Payment  : 2 , [np.float64(0.0), np.float64(1.0)]
 Total ERC20 tnxs  : 300 , [np.float64(0.0), np.float64(265.0)]
oldbalanceOrg  : 54092 , [np.float64(0.0), np.float64(19391.0)]
transaction_type_Deposit  : 2 , [np.float64(0.0), np.float64(1.0)]
newbalanceOrig  : 42997 , [np.float64(0.0), np.float64(16374.82)]
 ERC20_most_rec_token_type_Others  : 2 , [np.float64(0.0), np.float64(1.0)]
merchant_category_Utilities  : 2 , [np.float64(0.0), np.float64(1.0)]
 ERC20 total ether sent  : 1290 , [np.float64(0.0), np.float64(35603169.52)]
 ERC20 most sent token type_Golem  : 2 , [np.float64(0.0), np.float64(1.0)]
device_used_Web  : 2 , [np.float64(1.0), np.float64(-0.0)]
transaction_type_Bill Payment  : 2 , [np.float64(1.0), np.float64(0.0)]
transaction_type_Loan Disbursement  : 2 , [np.float64(0.0), np.float64(1.0)]
merchant_category_Groceries  : 2 , [np.float64(0.0), np.float64(1.0)]
 ERC

In [ ]:
print(a,b)

[2, 2, 300, 54092, 2, 42997, 2, 2, 1415, 2, 2, 2, 2, 2, 3460, 5854, 5717, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6728, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 4719, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2647, 3, 2, 2, 6647, 4, 123, 641, 5013, 1309, 6302, 2, 6767, 1130, 70, 101058, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5868, 4589, 4, 3380, 29, 61351, 57396, 476, 2, 1276, 2, 2] [2, 2, 300, 54092, 2, 42997, 2, 2, 1290, 2, 2, 2, 2, 2, 3190, 3903, 2942, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6194, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 2148, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2424, 3, 2, 2, 5277, 4, 123, 641, 5013, 1296, 4989, 2, 4029, 1112, 70, 101054, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5112, 2235, 4, 3113, 14, 61351, 57396, 444, 2, 900, 2, 2]


In [ ]:
print(b)

[2, 2, 300, 54092, 2, 42997, 2, 2, 1290, 2, 2, 2, 2, 2, 3190, 3903, 2942, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6194, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 2148, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2424, 3, 2, 2, 5277, 4, 123, 641, 5013, 1296, 4989, 2, 4029, 1112, 70, 101054, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5112, 2235, 4, 3113, 14, 61351, 57396, 444, 2, 900, 2, 2]


In [ ]:
print(imputed_data.isnull().sum())

isFraud                               0
transaction_type_EMI Payment          0
 Total ERC20 tnxs                     0
oldbalanceOrg                         0
transaction_type_Deposit              0
                                     ..
 ERC20 min val sent                   0
 ERC20 most sent token type_Others    0
 ERC20 min val rec                    0
type_CASH_OUT                         0
merchant_category_Entertainment       0
Length: 87, dtype: int64


In [ ]:
df2 = pd.concat([imputed_data, a[["TransactionID", "dataset_name"]]], axis=1)


In [ ]:
df2.to_csv("/content/drive/MyDrive/Assigments/vae_index_data.csv")

In [ ]:
imputed_data.to_csv("/content/drive/MyDrive/Assigments/vae_data_imputed_data.csv")

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'imputed_data' is your full DataFrame
train_df, test_df = train_test_split(imputed_data, test_size=0.2, random_state=42)

# Optional: Check the size
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (127872, 87)
Test shape: (31969, 87)


In [ ]:
train_df.to_csv("/content/drive/MyDrive/Assigments/vae_train.csv")
test_df.to_csv("/content/drive/MyDrive/Assigments/vae_test.csv")

In [ ]:
# import pandas as pd

# # Ensure pandas doesn't truncate the column output
# pd.set_option('display.max_rows', None)

# # Then print the null counts
# print(df2.isnull().sum())


isFraud                                              0
transaction_type_EMI Payment                         0
 Total ERC20 tnxs                                    0
oldbalanceOrg                                        0
transaction_type_Deposit                             0
newbalanceOrig                                       0
 ERC20_most_rec_token_type_Others                    0
merchant_category_Utilities                          0
 ERC20 total ether sent                              0
 ERC20 most sent token type_Golem                    0
device_used_Web                                      0
transaction_type_Bill Payment                        0
transaction_type_Loan Disbursement                   0
merchant_category_Groceries                          0
 ERC20 total Ether received                          0
avg val sent                                         0
total ether balance                                  0
max val sent to contract                             0
transactio

In [ ]:
imputed_data['merchant_fraud_Kilback LLC'].value_counts()

,count
"WALLSMATERIAL_MODE_Stone, brick",
0.211,50000
0.000,39446
1.000,10554


In [ ]:
imputed_data1=imputed_data.iloc[:,:30]

In [ ]:
imputed_data

In [ ]:
imputed_data1.to_csv('test.csv')

In [ ]:
imputed_data_round = imputed_data.round(0)
imputed_data_round

,amt,lat,long,city_pop,merch_lat,merch_long,merchant_fraud_Cormier LLC,merchant_fraud_Kilback LLC,merchant_fraud_Kuhn LLC,merchant_fraud_Schumm PLC,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Others,EMERGENCYSTATE_MODE_Yes,is_fraud
0,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.0,37.0,-91.0,241.0,36.0,-91.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,63.0,41.0,-74.0,5950.0,41.0,-74.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,52.0,27.0,-82.0,3776.0,26.0,-82.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,3.0,35.0,-99.0,540.0,35.0,-98.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
imputed_data_round['merchant_fraud_Kilback LLC'].value_counts()

,count
merchant_fraud_Kilback LLC,
0.0,99829
1.0,171


In [ ]:
imputed_data_round.to_csv('cred+simucred_vae.csv')

In [ ]:
dff=pd.read_csv("C:\Users\adars\final_data_vae.csv")

In [ ]:
for i in imputed_data_round.columns:
    print(i)

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

# Load your dataset
# df = pd.read_csv('your_dataset.csv')  # Load your DataFrame here if not already loaded

# Convert DataFrame to a numpy array
data = df_new.to_numpy()

# Convert to PyTorch tensors
data_tensor = torch.tensor(data, dtype=torch.float32)

# Create DataLoader
batch_size = 32
dataset = TensorDataset(data_tensor, data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# VAE Model
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        x_reconstructed = torch.sigmoid(self.fc2(h))
        return x_reconstructed

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, logvar

def vae_loss(reconstructed_x, x, mu, logvar):
    reconstruction_loss = F.mse_loss(reconstructed_x, x, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_divergence

# Training
input_dim = data.shape[1]
hidden_dim = 64
latent_dim = 2
vae = VAE(input_dim, hidden_dim, latent_dim)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

num_epochs = 100
vae.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        x, _ = batch
        optimizer.zero_grad()
        reconstructed_x, mu, logvar = vae(x)
        loss = vae_loss(reconstructed_x, x, mu, logvar)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Generating Synthetic Data
vae.eval()
with torch.no_grad():
    z = torch.randn(100, latent_dim)
    generated_samples = vae.decoder(z)
    generated_samples = generated_samples.numpy()
    print(generated_samples)


In [ ]:
data=df_new

In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Lambda
# from tensorflow.keras.models import Model
# from tensorflow.keras import backend as K
# from sklearn.preprocessing import MinMaxScaler

# # Example DataFrame
# # Replace this with your own dataset loading and preprocessing steps

# # Scale the data
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(df_new)

# # Define the VAE architecture
# original_dim = data.shape[1]
# intermediate_dim = 10
# latent_dim = 2
# batch_size = 10
# epochs = 50

# # Encoder
# inputs = Input(shape=(original_dim,))
# h = Dense(intermediate_dim, activation='relu')(inputs)
# z_mean = Dense(latent_dim)(h)
# z_log_var = Dense(latent_dim)(h)

# # Sampling function
# def sampling(args):
#     z_mean, z_log_var = args
#     epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
#     return z_mean + K.exp(0.5 * z_log_var) * epsilon

# # Reparameterization trick
# z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# # Decoder
# decoder_h = Dense(intermediate_dim, activation='relu')
# decoder_mean = Dense(original_dim, activation='sigmoid')
# h_decoded = decoder_h(z)
# x_decoded_mean = decoder_mean(h_decoded)

# # VAE model
# vae = Model(inputs, x_decoded_mean)

# # Loss function
# xent_loss = original_dim * tf.keras.losses.binary_crossentropy(inputs, x_decoded_mean)
# kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
# vae_loss = tf.reduce_mean(xent_loss + kl_loss)

# vae.add_loss(vae_loss)
# vae.compile(optimizer='adam')

# # Training
# vae.fit(data_scaled, epochs=epochs, batch_size=batch_size, verbose=0)

# # Impute missing values using the trained VAE
# data_imputed_scaled = vae.predict(data_scaled)

# # Inverse transform scaling
# data_imputed = scaler.inverse_transform(data_imputed_scaled)

# # Replace the missing values in original data with imputed values
# data_filled = data.copy()
# data_filled.values[missing_mask] = data_imputed[missing_mask]

# # Now data_filled contains your DataFrame with missing values imputed using VAE

# # Example of how to use the imputed data
# print("Original Data:")
# print(data)
# print("\nData with Missing Values:")
# print(data_missing)
# print("\nImputed Data:")
# print(data_filled)
